# Обучение модели с использованием PyTorch Elastic Learning

PyTorch Elastic Learning при обучении моделей позволяет масштабировать вычислительные ресурсы в зависимости от их доступности и повышает отказоустойчивость. Подробнее об использовании PyTorch Elastic Learning – в [документации ML Space](https://cloud.ru/docs/aicloud/mlspace/concepts/guides/guides__mt/environments__model-training__training-with-elastic-learning-example.html#pytorch-elastic-learning) и в [документации PyTorch](https://pytorch.org/docs/stable/elastic/run.html).

Чтобы не потерять результаты обучения при перезапуске задачи, в коде реализуют [сохранение чекпоинтов и запуск обучения с последнего чекпоинта](https://cloud.ru/docs/aicloud/mlspace/concepts/guides/guides__mt/environments__model-training__save-intermidiate-results.html). 

## 1. Подготовка

Импортируем библиотеки.

In [1]:
import client_lib
import pathlib

Устанавливаем переменные.

In [2]:
BASE_DIR = pathlib.Path().absolute()
print(f'Working dir: {BASE_DIR}')

Working dir: /home/jovyan/aicloud-examples/pytorch-elastic-example


# 2. Запуск задачи обучения

Запускать задачу в регионе будем с помощью класса `client_lib.Job()`. 

См. [Все способы запустить задачу обучения](https://cloud.ru/docs/aicloud/mlspace/concepts/guides/guides__mt/environments__model-training__training-process.html)


Для запуска укажем следующие параметры:
- `script`– путь к запускаемому скрипту;
- `base_image` – базовый образ, в котором будет исполняться скрипт обучения модели;
- `instance_type` – конфигурация вычислительных ресурсов, используемых для решения задач;
- `type` - тип распределённых вычислений;
- `n_workers` — количество рабочих узлов;
- `elastic_min_workers` — минимальное количество рабочих узлов;
- `elastic_max_workers` — максимальное количество рабочих узлов.

In [3]:
REGION = "SR006" # Region
INSTANCE_TYPE = "a100plus.1gpu.80vG.12C.96G" # Instance_type
N_WORKERS = 3
BASE_IMAGE = "cr.ai.cloud.ru/aicloud-base-images/cuda12.1-torch2-py39:0.0.36"

In [5]:
job_run = client_lib.Job(
    base_image=BASE_IMAGE,
    script=f'{BASE_DIR}/train_ddp_elastic_example-torch2.py',
    region=f'{REGION}',
    instance_type=f'{INSTANCE_TYPE}',
    flags={"epochs": 10},
    type="pytorch_elastic",
    n_workers=N_WORKERS,
    elastic_min_workers=1,
    elastic_max_workers=3,
    checkpoint_dir=f"{BASE_DIR}/logs",
)

In [6]:
job_run.submit()

'Job "lm-mpi-job-05c11349-feca-4ded-903f-f961c5eb3811" created.'

In [7]:
job_run.status()

'Job status=Completed'

In [8]:
job_run.logs()

2024-09-20T10:20:03Z master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
2024-09-20T10:20:03Z [2024-09-20 13:20:03,266] torch.distributed.launcher.api: [INFO] Starting elastic_operator with launch configs:
2024-09-20T10:20:03Z [2024-09-20 13:20:03,266] torch.distributed.launcher.api: [INFO]   entrypoint       : /home/jovyan/aicloud-examples/pytorch-elastic-example/train_ddp_elastic_example-torch2.py
2024-09-20T10:20:03Z [2024-09-20 13:20:03,266] torch.distributed.launcher.api: [INFO]   min_nodes        : 1
2024-09-20T10:20:03Z [2024-09-20 13:20:03,266] torch.distributed.launcher.api: [INFO]   max_nodes        : 3
2024-09-20T10:20:03Z [2024-09-20 13:20:03,266] torch.distributed.launcher.api: [INFO]   nproc_per_node   : 1
2024-09-20T10:20:03Z [2024-09-20 13:20:03,266] torch.distributed.launcher.api: [INFO]   run_id           : elastic
2024-09-20T10:20:03Z [2024-09-20 13:20:03,266] torch.distributed.launcher.api: [INFO]   rdzv_backend     : c10d
2024-